HSE, Applied Time Series Forecasitng , Fall 2024- Winter 2025

<font color="green"> Additional topic: Two Steps Cross Validation </font>

<span style="color:black; font-size: 12pt"></span>

Alexey Romanenko,
<font color="blue">alexromsput@gmail.com</font>

**Key words:**
two steps cross validation


In [58]:
import pandas as pd
import numpy as np
from random import SystemRandom, normalvariate
pd.options.plotting.backend = "plotly"

# Genereate random data set  with length L
# There is one target variable (random from Bernulli Distribution)
# There are d features: each of them is |random from standard Normal Distribution N(0,1)
L = 500
d = 1000

train = pd.DataFrame.from_records([
        [ SystemRandom().randint(0,1)]+ [normalvariate(0, 1) for j in range(d)]
        for i in range(L)
    ], columns = ['Label']+ ['F'+str(j)  for j in range(d)])

In [25]:
# let's have a look at data
train.head()

,Label,F0,F1,F2,F3,F4,F5,F6,F7,F8,...,F990,F991,F992,F993,F994,F995,F996,F997,F998,F999
0,0,-0.028648,-0.844419,1.919696,1.702614,1.387355,1.670022,-0.465732,1.151016,-0.272577,...,0.983089,-0.699517,-1.322571,-2.935829,0.324780,-1.440337,0.216241,-1.025063,-0.628220,-0.356698
1,0,-1.472261,-1.253820,2.347946,0.294180,0.175633,0.275891,1.664717,0.757378,-2.254533,...,1.443564,-1.581439,0.521923,0.245480,-1.837965,-0.735102,0.046520,-0.707016,-0.211181,1.005481
2,0,0.301961,0.621293,-0.052560,0.000748,-0.396068,-0.624181,-1.230741,0.270751,-0.113658,...,0.874385,-2.519516,0.471000,-1.268988,0.485241,-0.731311,-0.197811,-0.957912,0.032073,-0.934157
3,0,-0.459818,-2.472436,-0.886215,-0.588255,-0.634751,0.638416,-0.755783,-0.548295,-0.323629,...,-1.628672,0.657107,-0.589560,-0.025908,-0.289952,-0.091741,-0.232549,1.341281,-1.551488,-0.410126
4,0,-0.106088,0.224515,-0.251413,-1.321692,-0.015031,-0.947918,-0.350240,-0.754266,-1.049779,...,-0.601243,-1.996050,-2.093429,-1.502434,0.209881,0.741891,2.669829,1.023728,-0.853804,-0.376747


In [26]:
# let's have a look at data
train.groupby(['Label'])['Label'].count()

,Label
Label,
0,264
1,236


In [61]:
# actually each feature has got quite adequate distribution
train['Label'].hist()

In [62]:
# actually each feature has got quite adequate distribution
train['F11'].hist()

<font size = 5pt> Question </font>


**What is expected score for such dataset?**

 -  Answer:
    0.5 (i.e. 50%)

In [56]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

def qFold_accuracy(x,y, estimator, cv_strategy):
    score = 0
    for train_indices, test_indices in cv_strategy.split(x,y):
        estimator.fit(x[train_indices], y[train_indices])
        score += estimator.score(x[test_indices], y[test_indices])

    return score/cv_strategy.n_splits


# Train 1-feauture classifier
clf = KNeighborsClassifier(n_neighbors=1)
# clf = DecisionTreeClassifier()

max_score = 0
score_set = []
f=0
cv_k = 10
skf = StratifiedKFold(n_splits=cv_k, shuffle = True, random_state=1)
cv_strategy = skf.split(train[train.columns[1:]], train[train.columns[0]])

for j in range(1,d+1):       # feature number
    #    print('\n', '\n') #, kf_1

    scrrr  = qFold_accuracy(np.asarray(train[[train.columns[j]]]),
                            np.asarray(train[['Label']]).ravel(), clf, skf)
    score_set +=[scrrr]
    if(scrrr > max_score):
        max_score = scrrr
        f = j-1
print('k-Fold on score: ', max_score)
print('The Number of the best feature: ', f)

k-Fold on score:  0.5840000000000001
The Number of the best feature:  75


<font size = 5pt> NOTE CV score is highly above 0.5 </font>
<p>
It means that CV score doesn't show correct score for the model

In [54]:
# Lets generate a new dataset for assessement
test = pd.DataFrame.from_records([
        [ SystemRandom().randint(0,1)]+ [normalvariate(0, 1) for j in range(d)]
        for i in range(L)
    ], columns = ['Label']+ ['F'+str(j)  for j in range(d)])

# True score on Test Set
# use the same clissifier as previously
clf.fit(train[['F'+str(f)]], train['Label'])
true_score = accuracy_score(clf.predict(test[['F'+str(f)]]), test[['Label']])
print(f'{true_score: .3f}')

 0.502


<font size = 5pt> Score on independent test set is more close to 0.5 (== it's quite rare occasion that score on test set is much higher than 0.5)</font>
<p>
1. Check whether this result depends on classifier <p>
2. Check whether this result depends on the length of train set L<p>
3. Check whether this result depends on number of feature d

In [63]:
# distribution of score metric
pd.DataFrame(score_set).hist()